In [64]:
import numpy as np
import pandas as pd
from plotnine import *
from sko.SA import SA

In [58]:
df = pd.DataFrame({
    'sst': np.array([8.05, 7.85, 7.78, 7.56, 7.47, 7.76, 7.91, 8.34]),
    'lon': np.array([-0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5]),
})
df

,sst,lon
0,8.05,-0.5
1,7.85,1.5
2,7.78,2.5
3,7.56,3.5
4,7.47,4.5
5,7.76,5.5
6,7.91,6.5
7,8.34,7.5


In [59]:
from scipy.interpolate import interp1d

f = interp1d(df['lon'], df['sst'], kind=3)
lon_array = np.linspace(-0.5, 7.5, 50)
sst_array = f(lon_array)

df = pd.DataFrame({
    'sst': sst_array,
    'lon': lon_array
})
df

,sst,lon
0,8.050000,-0.500000
1,8.003419,-0.336735
2,7.965075,-0.173469
3,7.934132,-0.010204
4,7.909752,0.153061
5,7.891099,0.316327
6,7.877337,0.479592
7,7.867629,0.642857
8,7.861138,0.806122
9,7.857028,0.969388


In [61]:
(
    ggplot(df, aes('lon', 'sst', color='sst'))
    + geom_line()
    + geom_vline(aes(xintercept=3.5), linetype='--')
    + labs(x='Longitude',
           y='SST',
           title='Variation of SST from 0.5°E to 6.5°E in the 53°N Sea Area',
           color='SST')
).save('img/fish/variation of sst from 0.5°E to 6.5°E in the 53°N sea area.png')

c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: img/fish/variation of sst from 0.5°E to 6.5°E in the 53°N sea area.png


In [76]:
def gaussian(x, mu, sigma):
    return 1 / (np.sqrt(2*np.pi)*sigma) \
    * np.exp(-np.power(x-mu,2)/(2*np.power(sigma,2)))

def fish_density(lon, lon1, lon2):
    return gaussian(lon, lon1, np.sqrt(6))\
           + gaussian(lon, lon2, np.sqrt(6))

def best_reward(start_lon, move_time, lon1, lon2):
    if start_lon <= np.mean([lon1, lon2]):
        return 1/day * (8*day-move_time) * fish_density(start_lon+0.2*move_time, lon1, lon2)
    if start_lon > np.mean([lon1, lon2]):
        return 1/day * (8*day-move_time) * fish_density(start_lon+0.2*move_time, lon1, lon2)

In [117]:
def best_reward_for_sa(x):
    if x[0] < 0 or x[0] > 8 * day:
        return np.Inf
    if -.5 + 0.3288 * x[0] > 3.5:
        return np.Inf
    return -best_reward(-.5, x[0], 3.7, 4.6)

sa = SA(func=best_reward_for_sa,
        x0=[-.5])

result_dict = {}
for day in [1, 3, 5, 7]:
    x_result, y_result = sa.run()
    result_dict[day] = np.concatenate([x_result.reshape(-1,1),
                                       np.array(y_result).reshape(-1,1)]).flatten()

result_dict

{1: array([ 1.02402844, -0.45468692]),
 3: array([11.16282841, -0.85601662]),
 5: array([12.16535516, -1.20020519]),
 7: array([12.16544996, -1.35008551])}

In [120]:
df_temp = pd.DataFrame(columns=['t', 'reward'])
for result in result_dict:
    df_temp = pd.concat([df_temp, pd.DataFrame({
        't': [result_dict[result][0]],
        'reward': -np.array([result_dict[result][1]]) * 12 * 3179.1,
    })])
df_temp

,t,reward
0,1.024028,17345.942091
0,11.162828,32656.349149
0,12.165355,45786.867936
0,12.165450,51504.682017


In [121]:
df_with_border = df_temp.copy()
df_with_border['border'] = 'Have Border'
df_with_border

,t,reward,border
0,1.024028,17345.942091,Have Border
0,11.162828,32656.349149,Have Border
0,12.165355,45786.867936,Have Border
0,12.165450,51504.682017,Have Border


In [114]:
df_not_with_border = df_temp.copy()
df_not_with_border['border'] = 'Not Have Border'
df_not_with_border

,t,reward,border
0,1.024045,17345.942091,Not Have Border
0,11.162271,32656.349228,Not Have Border
0,16.612995,49591.327599,Not Have Border
0,19.048984,60934.370476,Not Have Border


In [126]:
df_temp = pd.concat([df_with_border, df_not_with_border])
df_temp = df_temp.reset_index(drop=True)
df_temp['day'] = ['1','3','5','7','1','3','5','7']
df_temp

,t,reward,border,day
0,1.024028,17345.942091,Have Border,1
1,11.162828,32656.349149,Have Border,3
2,12.165355,45786.867936,Have Border,5
3,12.165450,51504.682017,Have Border,7
4,1.024045,17345.942091,Not Have Border,1
5,11.162271,32656.349228,Not Have Border,3
6,16.612995,49591.327599,Not Have Border,5
7,19.048984,60934.370476,Not Have Border,7


In [127]:
(
    ggplot(df_temp, aes('day', 'reward', fill='border'))
    + geom_col(position='dodge')
    + labs(x='Time at Sea, [Day]',
           y='Reward, [Pounds]',
           title='the Relationship between Income and Borders',
           color='Border')
).save('img/fish/the relationship between income and borders.png')


c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\environment\python3.8.5\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: img/fish/the relationship between income and borders.png
